<a href="https://colab.research.google.com/github/GioDio1002/HuggingFace-Mini-Project/blob/main/chi/%E5%85%A8%E8%AF%8D%E9%81%AE%E7%BD%A9/%E6%9D%B0%E5%93%A5%E4%B8%8D%E8%A6%81_BERT_WMM_%E5%85%A8%E8%AF%8D%E9%81%AE%E7%BD%A9%E5%A1%AB%E5%85%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前言
最近在B站杰哥区看了几百集平行宇宙剧本，也看过114514次翻译和AI续写的版本。忽然，萌生了一个想法：**如果把200多句对白部分的字词遮罩，让AI去填写会有什么效果呢?**。*我很好奇*，因此本代码主要以全词遮罩的中文预训练模型BERT-wwm去探索"杰哥不要"的**可能性**。

# 参考
[HuggingFace Transformer API](https://huggingface.co/hfl/chinese-bert-wwm-ext)<br>
[中文BERT-wwm](https://github.com/ymcui/Chinese-BERT-wwm)<br>
[杰哥不要啦完整台词整理](https://www.bilibili.com/read/cv9034147/)<br>
[BERT模型 vs 王思聪！看一看谁的土味情话更加肉麻？](https://mp.weixin.qq.com/s/6QE2CfFwNgMawZ7JdtX19g)<br>
[用Python制作酷炫词云图，原来这么简单！
](https://blog.csdn.net/weixin_46089319/article/details/104009532)

代码开发环境为Google Colab,所以使用不同环境的朋友们注意要对应library的安装

In [ ]:
#安装 Transformers
!pip install transformers

     |████████████████████████████████| 2.5MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 41.4MB/s 
     |████████████████████████████████| 901kB 37.9MB/s 


In [ ]:
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import bleach
import random
from tqdm.notebook import tqdm

# 数据收集

这里感谢up主碳酸镁儿的整理，这里主要是关键对白，本人在这尽量作补充和修正，日后有我可能尝试*python从影片抽取字幕的项目*。

In [ ]:
#简单爬虫
response = requests.get(url='https://www.bilibili.com/read/cv9034147/')
response.encoding = response.apparent_encoding
soup = BeautifulSoup(response.content, 'html.parser')
para = soup.find_all('p')  #Get the paragraph element
para

[<p class="inner-title" title="杰哥不要啦完整台词整理">杰哥不要啦完整台词整理</p>,
 <p>阿伟：弱耶，拜托你很弱耶，你现在知道谁是老大了喔</p>,
 <p>阿妈：阿伟你又在打电动喔，休息一下吧，去看看书好不好</p>,
 <p>阿伟：烦呐</p>,
 <p>阿妈：我在跟你讲话，有没有听到</p>,
 <p>阿伟：你不要烦好不好</p>,
 <p>阿妈：我才说你两句你就说我烦，我只希望你能够好好用功读书，整天只看到你在这边打电动</p>,
 <p>阿伟：靠，输了啦，都是你害的啦，拜托</p>,
 <p>阿伟：他超废的</p>,
 <p>彬彬：我知道啊，之前我就干掉他过</p>,
 <p>阿伟：他每次都被我洗战绩耶</p>,
 <p>彬彬：他就嫩啊</p>,
 <p>    杰哥回眸</p>,
 <p>阿伟：快点啦，我看到了，左边左边</p>,
 <p>彬彬：好了</p>,
 <p>阿伟：上面上面……左边左边</p>,
 <p>彬彬：你不会放诶</p>,
 <p>               我输了啊，快点放…</p>,
 <p>               我输了，不要玩了</p>,
 <p>阿伟：诶，你不要每次都这样好不好</p>,
 <p>彬彬：不想玩了啦</p>,
 <p>阿伟：你还有没有钱啊，我肚子好饿</p>,
 <p>彬彬：没了啦，今天都花光了</p>,
 <p>阿伟：我刚刚不是叫你多带一点吗</p>,
 <p>彬彬：你干嘛自己不多带</p>,
 <p>阿伟：你不要每次都这样好不好啦</p>,
 <p>彬彬：好啦，时间也差不多了</p>,
 <p>杰哥在等他们</p>,
 <p>阿伟：彬彬，我好饿喔</p>,
 <p>彬彬：我们两个都没钱了</p>,
 <p>阿伟：你要干嘛</p>,
 <p>彬彬：没有钱我们只能回家</p>,
 <p>阿伟：拜托，我才不要回家咧，我阿妈超凶的耶，去住你家啦</p>,
 <p>彬彬：不行啊</p>,
 <p>阿伟：为什么不行</p>,
 <p>彬彬：我自己都自身难保了，而且我爸会揍我</p>,
 <p>（此时阿杰出现）</p>,
 <p>阿杰：不好意思，我刚听到你们两个说肚子饿，我这里刚好有个面包，给你们吃（显示自己的力量，表示愿意帮助二人），我还不饿，来，请你们吃（表达善意）</p

# 数据清洗

In [ ]:
para1,script = [],[]
for p in para[1:-1]: #只关注剧本内容
  para1.append(str(p))  #转换格式
for p in para1:
    tmp = bleach.clean(p,tags=[],strip=True).replace('\xa0','').replace(' ','').replace('萎','伟')#去掉tag和脏数据，同步角色名称
    if  not  '：'   in tmp and not  '（' in  tmp :#标识动作和过场，注意是中文的括号
      tmp = '（' +tmp+'）'
    script.append(tmp)
script

['阿伟：弱耶，拜托你很弱耶，你现在知道谁是老大了喔',
 '阿妈：阿伟你又在打电动喔，休息一下吧，去看看书好不好',
 '阿伟：烦呐',
 '阿妈：我在跟你讲话，有没有听到',
 '阿伟：你不要烦好不好',
 '阿妈：我才说你两句你就说我烦，我只希望你能够好好用功读书，整天只看到你在这边打电动',
 '阿伟：靠，输了啦，都是你害的啦，拜托',
 '阿伟：他超废的',
 '彬彬：我知道啊，之前我就干掉他过',
 '阿伟：他每次都被我洗战绩耶',
 '彬彬：他就嫩啊',
 '（杰哥回眸）',
 '阿伟：快点啦，我看到了，左边左边',
 '彬彬：好了',
 '阿伟：上面上面……左边左边',
 '彬彬：你不会放诶',
 '（我输了啊，快点放…）',
 '（我输了，不要玩了）',
 '阿伟：诶，你不要每次都这样好不好',
 '彬彬：不想玩了啦',
 '阿伟：你还有没有钱啊，我肚子好饿',
 '彬彬：没了啦，今天都花光了',
 '阿伟：我刚刚不是叫你多带一点吗',
 '彬彬：你干嘛自己不多带',
 '阿伟：你不要每次都这样好不好啦',
 '彬彬：好啦，时间也差不多了',
 '（杰哥在等他们）',
 '阿伟：彬彬，我好饿喔',
 '彬彬：我们两个都没钱了',
 '阿伟：你要干嘛',
 '彬彬：没有钱我们只能回家',
 '阿伟：拜托，我才不要回家咧，我阿妈超凶的耶，去住你家啦',
 '彬彬：不行啊',
 '阿伟：为什么不行',
 '彬彬：我自己都自身难保了，而且我爸会揍我',
 '（此时阿杰出现）',
 '阿杰：不好意思，我刚听到你们两个说肚子饿，我这里刚好有个面包，给你们吃（显示自己的力量，表示愿意帮助二人），我还不饿，来，请你们吃（表达善意）',
 '阿伟（小声）：先吃先吃',
 '阿杰：对了，我叫阿杰，我也常来这里玩，他们都叫我杰哥，你们好，我一个人住，我的房子还蛮大的，欢迎你们来我家玩。玩累了就直接睡觉，没问题的。',
 '阿伟：你觉得咧？',
 '彬彬：我觉得怪怪的。',
 '阿伟：看起来就是个很奇怪的人啊',
 '彬彬：不要理他',
 '阿杰：我常常帮助一些翘家的人，如果你们不要来的话，也没有关系；如果要来的话，我等一下可以带你们去超商，买一些好吃的喔。',
 '阿伟：有东西可以吃耶，要不要去啊？',
 '彬彬：好啊，不然去好了。',
 '阿伟：那杰哥，我跟我朋友今天

# 数据探索

理论上是应该清洗前探索的，反正都~~全文背诵~~，日后有机会补上词云图吧





# 特征工程
遮蔽的效果是随机，目前模型只允许文本有一个[MASK],有两种方案：
1.   按标点符号分句，随机进行文本遮罩
2.   每个人发言，一整段包括标点符号随机[MASK]

尽管BERT-WMM模型能够预测标点符号，但为了效果，还是使用方案一


In [ ]:
def random_mask(script):
  r1 = []
  for s1 in script:
      ls = s1.split('，') #按逗号分句
      r2 = ''
      for s2 in ls: #随机遮罩
        if '：' in s2:
            n = len(s2[2:])
            index = random.randint(3,n+1)
            r2 += s2[:index]+'[MASK]'+s2[index+1:]+'，'
        elif 'dua' in s2: #针对登dua郎中的拼音
            r2 += s2.replace('dua','[MASK]')
        else:
            n = len(s2)
            index = random.randint(0,n)
            r2 += s2[:index]+'[MASK]'+s2[index+1:]+'，'
      if r2.endswith('，'): #去掉最后的逗号
          r2 = r2[:-1]
      r1.append(r2) 
  return r1

In [ ]:
masked = random_mask(script)
masked

['阿伟：[MASK]耶，拜托你[MASK]弱耶，你[MASK]在知道谁是老大了喔',
 '阿妈：阿伟你又在打电动[MASK]，休息[MASK]下吧，[MASK]看看书好不好',
 '阿伟：烦[MASK]',
 '阿妈：[MASK]在跟你讲话，有没有听到[MASK]',
 '阿伟：你不要烦[MASK]不好',
 '阿妈：[MASK]才说你两句你就说我烦，我[MASK]希望你能够好好用功读书，整天只看[MASK]你在这边打电动',
 '阿伟：[MASK]，输了啦[MASK]，都是你害的啦[MASK]，拜托[MASK]',
 '阿伟：他[MASK]废的',
 '彬彬：我[MASK]道啊，之前我就干掉[MASK]过',
 '阿伟：他[MASK]次都被我洗战绩耶',
 '彬彬：他就[MASK]啊',
 '[MASK]杰哥回眸）',
 '阿伟：快[MASK]啦，我[MASK]到了，左边左边[MASK]',
 '彬彬：[MASK]了',
 '阿伟：上面上面……[MASK]边左边',
 '彬彬：你不[MASK]放诶',
 '（我输[MASK]啊，快点放[MASK]）',
 '（我[MASK]了，不要[MASK]了）',
 '阿伟：[MASK]，你不[MASK]每次都这样好不好',
 '彬彬：不想玩[MASK]啦',
 '阿伟：[MASK]还有没有钱啊，我肚子好[MASK]',
 '彬彬：没[MASK]啦，今天都花[MASK]了',
 '阿伟：我刚刚不是叫你[MASK]带一点吗',
 '彬彬：你干嘛自己[MASK]多带',
 '阿伟：你不[MASK]每次都这样好不好啦',
 '彬彬：好[MASK]，时间也差不多[MASK]',
 '[MASK]杰哥在等他们）',
 '阿伟：[MASK]彬，[MASK]好饿喔',
 '彬彬：我们两个都[MASK]钱了',
 '阿伟：你[MASK]干嘛',
 '彬彬：没有钱我们只能[MASK]家',
 '阿伟：[MASK]托，我才不要回家[MASK]，我阿[MASK]超凶的耶，去住你[MASK]啦',
 '彬彬：[MASK]行啊',
 '阿伟：为什么不[MASK]',
 '彬彬：我自[MASK]都自身难保了，而[MASK]我爸会揍我',
 '（此时阿杰出现[MASK]',
 '阿杰：[MASK]好意思，我刚[MASK]到你们两个说肚子饿，我这里刚好有个

# 建模及预测

这次只是调用BERT的预训练模型，所以没有训练过程。直接让我康康预测效果！

In [ ]:
#定义pipeline作预测
unmasker = pipeline('fill-mask', model='bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
exp = unmasker('阿伟/彬彬：[MASK]啊，走走走') #测试输出
exp

[{'score': 0.474282443523407,
  'sequence': '阿 伟 / 彬 彬 ： 是 啊 ， 走 走 走',
  'token': 3221,
  'token_str': '是'},
 {'score': 0.2970081865787506,
  'sequence': '阿 伟 / 彬 彬 ： 好 啊 ， 走 走 走',
  'token': 1962,
  'token_str': '好'},
 {'score': 0.07564515620470047,
  'sequence': '阿 伟 / 彬 彬 ： 对 啊 ， 走 走 走',
  'token': 2190,
  'token_str': '对'},
 {'score': 0.02331240475177765,
  'sequence': '阿 伟 / 彬 彬 ： 行 啊 ， 走 走 走',
  'token': 6121,
  'token_str': '行'},
 {'score': 0.018891604617238045,
  'sequence': '阿 伟 / 彬 彬 ： 走 啊 ， 走 走 走',
  'token': 6624,
  'token_str': '走'}]

我们可以看到候选词是有对应几率，所以之后会按分数，记录三个最高分数版本的BERT生成文本

In [ ]:
def mask_prediction(masked,prob):
  r1 = []
  for s1 in tqdm(masked):
    r2 = ''
    for s2 in s1.split('，'):
      exp = unmasker(s2)
      r2 += exp[prob]['sequence']+'，'
    if r2.endswith('，'): #去掉最后的逗号
        r2 = r2[:-1]
    r1.append(r2)
  return r1

In [ ]:
first = mask_prediction(masked,prob=0) #最高概率文本
first 

['阿 伟 ： 哈 耶，拜 托 你 好 弱 耶，你 现 在 知 道 谁 是 老 大 了 喔',
 '阿 妈 ： 阿 伟 你 又 在 打 电 动 。，休 息 一 下 吧，来 看 看 书 好 不 好',
 '阿 伟 ： 烦 。',
 '阿 妈 ： 我 在 跟 你 讲 话，有 没 有 听 到 ？',
 '阿 伟 ： 你 不 要 烦 我 不 好',
 '阿 妈 ： 我 才 说 你 两 句 你 就 说 我 烦，我 也 希 望 你 能 够 好 好 用 功 读 书，整 天 只 看 到 你 在 这 边 打 电 动',
 '阿 伟 ： ？，输 了 啦 ！，都 是 你 害 的 啦 ！，拜 托 ！',
 '阿 伟 ： 他 是 废 的',
 '彬 彬 ： 我 知 道 啊，之 前 我 就 干 掉 了 过',
 '阿 伟 ： 他 每 次 都 被 我 洗 战 绩 耶',
 '彬 彬 ： 他 就 是 啊',
 '（ 杰 哥 回 眸 ）',
 '阿 伟 ： 快 点 啦，我 看 到 了，左 边 左 边 。',
 '彬 彬 ： 走 了',
 '阿 伟 ： 上 面 上 面 右 边 左 边',
 '彬 彬 ： 你 不 要 放 诶',
 '（ 我 输 了 啊，快 点 放 下 ）',
 '（ 我 哭 了，不 要 哭 了 ）',
 '阿 伟 ： ？，你 不 要 每 次 都 这 样 好 不 好',
 '彬 彬 ： 不 想 玩 了 啦',
 '阿 伟 ： 你 还 有 没 有 钱 啊，我 肚 子 好 ！',
 '彬 彬 ： 没 有 啦，今 天 都 花 光 了',
 '阿 伟 ： 我 刚 刚 不 是 叫 你 多 带 一 点 吗',
 '彬 彬 ： 你 干 嘛 自 己 吃 多 带',
 '阿 伟 ： 你 不 要 每 次 都 这 样 好 不 好 啦',
 '彬 彬 ： 好 。，时 间 也 差 不 多 。',
 '（ 杰 哥 在 等 他 们 ）',
 '阿 伟 ： 阿 彬，我 好 饿 喔',
 '彬 彬 ： 我 们 两 个 都 有 钱 了',
 '阿 伟 ： 你 在 干 嘛',
 '彬 彬 ： 没 有 钱 我 们 只 能 搬 家',
 '阿 伟 ： 拜 托，我 才 不 要 回 家 。，我 阿 姨 超 凶 的 耶，去 住 你 家 啦',
 '彬 彬 ： 不 行 啊',
 '阿 伟 ： 为 什 么 不 ？',
 '彬 彬 ： 我 

In [ ]:
second = mask_prediction(masked,prob=1) #第二高概率文本
second

['阿 伟 ： 哈 耶，拜 托 你 好 弱 耶，你 现 在 知 道 谁 是 老 大 了 喔',
 '阿 妈 ： 阿 伟 你 又 在 打 电 动 。，休 息 一 下 吧，来 看 看 书 好 不 好',
 '阿 伟 ： 烦 。',
 '阿 妈 ： 我 在 跟 你 讲 话，有 没 有 听 到 ？',
 '阿 伟 ： 你 不 要 烦 我 不 好',
 '阿 妈 ： 我 才 说 你 两 句 你 就 说 我 烦，我 也 希 望 你 能 够 好 好 用 功 读 书，整 天 只 看 到 你 在 这 边 打 电 动',
 '阿 伟 ： ？，输 了 啦 ！，都 是 你 害 的 啦 ！，拜 托 ！',
 '阿 伟 ： 他 是 废 的',
 '彬 彬 ： 我 知 道 啊，之 前 我 就 干 掉 了 过',
 '阿 伟 ： 他 每 次 都 被 我 洗 战 绩 耶',
 '彬 彬 ： 他 就 是 啊',
 '（ 杰 哥 回 眸 ）',
 '阿 伟 ： 快 点 啦，我 看 到 了，左 边 左 边 。',
 '彬 彬 ： 走 了',
 '阿 伟 ： 上 面 上 面 右 边 左 边',
 '彬 彬 ： 你 不 要 放 诶',
 '（ 我 输 了 啊，快 点 放 下 ）',
 '（ 我 哭 了，不 要 哭 了 ）',
 '阿 伟 ： ？，你 不 要 每 次 都 这 样 好 不 好',
 '彬 彬 ： 不 想 玩 了 啦',
 '阿 伟 ： 你 还 有 没 有 钱 啊，我 肚 子 好 ！',
 '彬 彬 ： 没 有 啦，今 天 都 花 光 了',
 '阿 伟 ： 我 刚 刚 不 是 叫 你 多 带 一 点 吗',
 '彬 彬 ： 你 干 嘛 自 己 吃 多 带',
 '阿 伟 ： 你 不 要 每 次 都 这 样 好 不 好 啦',
 '彬 彬 ： 好 。，时 间 也 差 不 多 。',
 '（ 杰 哥 在 等 他 们 ）',
 '阿 伟 ： 阿 彬，我 好 饿 喔',
 '彬 彬 ： 我 们 两 个 都 有 钱 了',
 '阿 伟 ： 你 在 干 嘛',
 '彬 彬 ： 没 有 钱 我 们 只 能 搬 家',
 '阿 伟 ： 拜 托，我 才 不 要 回 家 。，我 阿 姨 超 凶 的 耶，去 住 你 家 啦',
 '彬 彬 ： 不 行 啊',
 '阿 伟 ： 为 什 么 不 ？',
 '彬 彬 ： 我 

In [ ]:
third = mask_prediction(masked,prob=3) #第三高概率文本
third

['阿 伟 ： 耶 耶，拜 托 你 很 弱 耶，你 都 在 知 道 谁 是 老 大 了 喔',
 '阿 妈 ： 阿 伟 你 又 在 打 电 动...，休 息 ㄧ 下 吧，多 看 看 书 好 不 好',
 '阿 伟 ： 烦...',
 '阿 妈 ： 是 在 跟 你 讲 话，有 没 有 听 到 ，',
 '阿 伟 ： 你 不 要 烦 了 不 好',
 '阿 妈 ： 他 才 说 你 两 句 你 就 说 我 烦，我 只 希 望 你 能 够 好 好 用 功 读 书，整 天 只 看 看 你 在 这 边 打 电 动',
 '阿 伟 ：...，输 了 啦 ，，都 是 你 害 的 啦 ，，拜 托 ，',
 '阿 伟 ： 他 要 废 的',
 '彬 彬 ： 我 难 道 啊，之 前 我 就 干 掉 她 过',
 '阿 伟 ： 他 几 次 都 被 我 洗 战 绩 耶',
 '彬 彬 ： 他 就 在 啊',
 '张 杰 哥 回 眸 ）',
 '阿 伟 ： 快 乐 啦，我 找 到 了，左 边 左 边...',
 '彬 彬 ： 来 了',
 '阿 伟 ： 上 面 上 面 上 边 左 边',
 '彬 彬 ： 你 不 会 放 诶',
 '（ 我 输 球 啊，快 点 放 假 ）',
 '（ 我 走 了，不 要 說 了 ）',
 '阿 伟 ：...，你 不 想 每 次 都 这 样 好 不 好',
 '彬 彬 ： 不 想 玩 啊 啦',
 '阿 伟 ： 他 还 有 没 有 钱 啊，我 肚 子 好 ，',
 '彬 彬 ： 没 用 啦，今 天 都 花 錢 了',
 '阿 伟 ： 我 刚 刚 不 是 叫 你 们 带 一 点 吗',
 '彬 彬 ： 你 干 嘛 自 己 想 多 带',
 '阿 伟 ： 你 不 用 每 次 都 这 样 好 不 好 啦',
 '彬 彬 ： 好 ，，时 间 也 差 不 多 ，',
 '杰 杰 哥 在 等 他 们 ）',
 '阿 伟 ： 郑 彬，「 好 饿 喔',
 '彬 彬 ： 我 们 两 个 都 赚 钱 了',
 '阿 伟 ： 你 来 干 嘛',
 '彬 彬 ： 没 有 钱 我 们 只 能 卖 家',
 '阿 伟 ： 阿 托，我 才 不 要 回 家 ，，我 阿 伯 超 凶 的 耶，去 住 你 的 啦',
 '彬 彬 ： 我 行 啊',
 '阿 伟 ： 为 什 么 不?',
 '彬 彬 

人工对比过原文和三篇生成文章，发现以下三个特点:


1.   如果遮罩字词没有前后的字词，如:"[MASK]，",最高概率文本填充为"?"和"..."
2.   不能预测日常称呼，如:"杰[MASK]，",最高概率文本填充为"。",理论是"哥"/"弟"/~~"姐"~~

3.   目前按标点分句的结果不理想，可能是因为短句太难推理出全文，如: '**我**一个人住，我的**房子**还蛮大的'->'**有** 一 个 人 住，我 的 **肚 子** 还 蛮 大 的'<br>

总而言之，如果想要胡说八道，直接取最小概率的预测字词做文本遮罩填充便可。相反，倘若追求上文下理的文笔通顺，目前全文生成比遮罩预测更有效。





# 保存文本

In [ ]:
def save_txt(filename, data):
    file = open(filename,'a') 
    for d in tqdm(data): 
        file.writelines(d+'\n') 
    file.close() 

In [ ]:
path = '/content/drive/MyDrive/NLP_Project/sub/'
save_txt(path+'JiegeStop-BERT-WMM-1st.txt',first)
save_txt(path+'JiegeStop-BERT-WMM-2nd.txt',second)
save_txt(path+'JiegeStop-BERT-WMM-3rd.txt',third)

# 结论
使用第三高分数作填充字词还是挺鬼畜，例如:''阿 杰 ： 好 想 懂，我 房 里 有 一 些 尼 康 的'和 '（ 我 输 球 啊，快 点 放 假 ）'等，有兴趣的小伙伴们可以用来创作。

预训练模型是有极限的，并没有突破碳基生物的思维框架，如果有非碳基生物的文本作训练，或者在鬼畜区能带来意想不到的可能性。